# Before Building Model

In [ ]:
import numpy as np
import pandas as pd
import gc

from fastai.tabular.all import *

TRAIN_PATH = '../input/ubiquant-trainfeather/train.feather'
TARGET = "target"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything()

MODEL_VALID_PCT = 0.1
MODEL_CONT_NAMES = [f'f_{i}' for i in range(300)]
MODEL_BS = 4096
MODEL_LAYERS = [300, 100, 30]
MODEL_MONITOR = "valid_loss"
MODEL_EPOCHS = 5

train = pd.read_feather(TRAIN_PATH)
train.head()

# Build Model

In [ ]:
randomSplitter = RandomSplitter(valid_pct=MODEL_VALID_PCT)(range_of(train))
tabularPandas = TabularPandas(
    train,
    y_names=TARGET,
    cont_names=MODEL_CONT_NAMES,
    splits=randomSplitter
)
dataloaders = tabularPandas.dataloaders(bs=MODEL_BS)

dataloaders.show_batch()

learn = tabular_learner(dataloaders, layers=MODEL_LAYERS)
learn.lr_find()

learn.fit_one_cycle(MODEL_EPOCHS, cbs=[SaveModelCallback(monitor=MODEL_MONITOR, comp=np.less)])

del train
gc.collect()

# After Building Model

In [ ]:
import ubiquant
env = ubiquant.make_env()   
iter_test = env.iter_test()    
for (test_df, sample_prediction_df) in iter_test:
    dl = learn.dls.test_dl(test_df)
    y, _ = learn.get_preds(dl=dl)
    sample_prediction_df[TARGET] = y  
    env.predict(sample_prediction_df)   